In [34]:
import os, time
import chromadb

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.llms import Ollama
from langchain_core.runnables import RunnablePassthrough, RunnablePick
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

from langchain import hub

from chromadb.errors import InvalidDimensionException

In [33]:
DATABASE_PATH = '/home/raj/nlp/cmu-rag/rag/chroma/txt/'
# embedding_name = 'llama2'
# persist_directory = DATABASE_PATH + embedding_name
# embedding = OllamaEmbeddings(model=embedding_name)
# vector_store = Chroma(persist_directory=persist_directory, embedding_function=embedding)

EMBEDDING_OPTIONS = ['tinyllama', 'llama2', 'gemma', 'mistral', 'neural-chat', 'openchat']
VECTOR_STORE_DIRECTORIES = [DATABASE_PATH + embedding_name for embedding_name in EMBEDDING_OPTIONS]

QUESTION_CATEGORIES = ['history']
ANNOTATION_DIR = '/home/raj/nlp/cmu-rag/annotation/test/'


In [35]:
def load_vector_store(dir, embedding_name):
    try:
        vector_store = Chroma(persist_directory=dir, embedding_function=OllamaEmbeddings(model=embedding_name))
    except InvalidDimensionException:
        vector_store = Chroma(persist_directory=dir, embedding_function=OllamaEmbeddings(model=embedding_name), force=True)
    return vector_store

def load_vector_store_non_ollama_embedding(dir, embedding_model):
    try:
        vector_store = Chroma(persist_directory=dir, embedding_function=embedding_model)
    except InvalidDimensionException:
        vector_store = Chroma(persist_directory=dir, embedding_function=embedding_model, force=True)
    return vector_store

def create_chain(vector_store, llm_model = 'llama2'):
    rag_prompt_llama = hub.pull("rlm/rag-prompt-llama")
    prompt_message = """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use as few words as possible and keep the answer concise. Do not mention the context in your response.
    Question: {question} 
    Context: {context} 
    Answer:"""

    rag_prompt_llama.messages[0].prompt.template = prompt_message

    llm = Ollama(model = llm_model)

    def format_docs(docs):
        return "\n\n".join(doc.page_content for doc in docs)

    retriever = vector_store.as_retriever()

    qa_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | rag_prompt_llama
        | llm
        | StrOutputParser()
    )

    return qa_chain


In [36]:
def get_questions(category='', dir = ANNOTATION_DIR):
    questions = []
    for file in os.listdir(dir + category):
        if file.endswith('questions.txt'):
            with open(dir + category + '/' + file, 'r') as f:
                for line in f.readlines():
                    questions.append(line.strip())
    return questions

def generate_answers(qa_chain, questions):
    if not questions:
        raise ValueError("No questions to answer")
    if not qa_chain:
        raise ValueError("No qa_chain to answer questions")
    answers = []
    for question in questions:
        if not question:
            continue
        answer = dict()
        answer_raw = qa_chain.invoke(question)
        answer["raw"] = answer_raw
        num_lines = answer_raw.count('\n')
        answer["num_lines"] = num_lines
        lines = answer_raw.split('\n')
        if num_lines == 0:
            answer["processed"] = lines[0]
        else:
            answer_lines = []
            for line in lines:
                if "i don't know" not in line.lower():
                    answer_lines.append(line)
            answer["processed"] = " ".join(answer_lines)
        answers.append(answer)

    return answers

def write_answers(answers, file_name, append = False):
    with open(file_name, 'a' if append else 'w') as f:
        for answer in answers:
            f.write(answer["processed"] + '\n')


In [38]:
vector_stores, question_categories = ['llama2', 'bge-large-en'], ['history']
# vector_stores, question_categories = VECTOR_STORE_DIRECTORIES, QUESTION_CATEGORIES
embedding_models = [OllamaEmbeddings(model='llama2'), 
                    HuggingFaceBgeEmbeddings(model_name="BAAI/bge-large-en", model_kwargs={"device": "cuda"}, encode_kwargs={"normalize_embeddings": True})
                    ]


for dir in vector_stores:
    embedding_name = 'llama2'
    # vector_store = load_vector_store(dir, embedding_name)
    vector_store = load_vector_store_non_ollama_embedding(dir, embedding_models[vector_stores.index(dir)])
    chain = create_chain(vector_store, embedding_name)
    for category in question_categories:
        print("Answering questions for category: {} with embedding {}".format(category, embedding_name))
        questions = get_questions(category, dir=ANNOTATION_DIR)
        answers = generate_answers(qa_chain=chain, questions=questions)
        write_to_file = embedding_name + '_answers.txt'
        write_answers(answers, ANNOTATION_DIR + category + '/' + write_to_file)
        print(f"Answers written to {ANNOTATION_DIR + category + '/' + write_to_file}")

Answering questions for category: history with embedding llama2
Answers written to /home/raj/nlp/cmu-rag/annotation/test/history/llama2_answers.txt
Answering questions for category: history with embedding llama2
Answers written to /home/raj/nlp/cmu-rag/annotation/test/history/llama2_answers.txt


In [15]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-large-en"
model_kwargs = {"device": "cuda"}
encode_kwargs = {"normalize_embeddings": True}
hf = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)


import torch

#clear cache on cuda
torch.cuda.empty_cache()

embedding_name = 'bge-large-en'
vector_store = load_vector_store_non_ollama_embedding(dir=DATABASE_PATH+'bge-large-en', embedding_model=hf)
chain = create_chain(vector_store)
questions = get_questions(category, dir=ANNOTATION_DIR)
answers = generate_answers(qa_chain=chain, questions=questions)
write_to_file = embedding_name + '_answers.txt'
write_answers(answers, ANNOTATION_DIR + category + '/' + write_to_file)


NameError: name 'category' is not defined

In [30]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-large-en"
model_kwargs = {"device": "cuda"}
encode_kwargs = {"normalize_embeddings": True}
hf = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)


import torch

#clear cache on cuda
torch.cuda.empty_cache()

TEST_QUESTION_PATH='/home/raj/nlp/cmu-rag/rveerara/system_outputs/'
embedding_name = 'bge-large-en'
vector_store = load_vector_store_non_ollama_embedding(dir=DATABASE_PATH+'bge-large-en', embedding_model=hf)
chain = create_chain(vector_store)
questions = get_questions(dir=TEST_QUESTION_PATH)
print(questions)

for i in range(0, len(questions), 10):
    question_set = questions[i:min(i+10, len(questions))]
    answers = generate_answers(qa_chain=chain, questions=question_set)
    write_to_file = TEST_QUESTION_PATH + 'system_output_1.txt'
    print("writing to file for questions: ", i, " to ", i+10, " to file: ", write_to_file)
    write_answers(answers, write_to_file, append=True)

writing to file for questions:  0  to  10  to file:  /home/raj/nlp/cmu-rag/rveerara/system_outputs/system_output_1.txt
writing to file for questions:  10  to  20  to file:  /home/raj/nlp/cmu-rag/rveerara/system_outputs/system_output_1.txt
writing to file for questions:  20  to  30  to file:  /home/raj/nlp/cmu-rag/rveerara/system_outputs/system_output_1.txt
writing to file for questions:  30  to  40  to file:  /home/raj/nlp/cmu-rag/rveerara/system_outputs/system_output_1.txt
writing to file for questions:  40  to  50  to file:  /home/raj/nlp/cmu-rag/rveerara/system_outputs/system_output_1.txt
writing to file for questions:  50  to  60  to file:  /home/raj/nlp/cmu-rag/rveerara/system_outputs/system_output_1.txt
writing to file for questions:  60  to  70  to file:  /home/raj/nlp/cmu-rag/rveerara/system_outputs/system_output_1.txt
writing to file for questions:  70  to  80  to file:  /home/raj/nlp/cmu-rag/rveerara/system_outputs/system_output_1.txt
writing to file for questions:  80  to  9